In [1]:
from typing import List

import numpy as np
import scipy.sparse as sp

# Project 4: Spectral clustering users based on their preferences (50 pt)

The goal of this task is to find groups of users with similar preferences using **Spectral clustering**. 
You are given a fragment of the Yelp social network, represented by an undirected weighted graph.
Nodes in the graph represent users.
If two users are connected by an edge of weight $w$, it means that they have both left positive reviews to the same $w$ restaurants.

Additionally, you are given a matrix `F` that encodes user preferences to different categories of restaurants. If `F[i, c] = 1`, then user `i` likes restaurants in category `c`.

You are allowed to use the imported functions (`eigsh`, `KMeans`, `normalize`).

## General remarks
Do not add or modify any code outside of the following comment blocks, or where otherwise explicitly stated.

``` python
##########################################################
# YOUR CODE HERE
...
##########################################################
```
After you fill in all the missing code, restart the kernel and re-run all the cells in the notebook.

The following things are **NOT** allowed:
- Using additional `import` statements
- Copying / reusing code from other sources (e.g. code by other students)

If you plagiarise even for a single project task, you won't be eligible for the bonus this semester.

## Load the data

* `N` = number of users (nodes in the graph)
* `C` = number of categories
* The graph is stored as a _sparse adjacency matrix_ `A` (shape `[N, N]`).
* User preferences are stored in a _feature matrix_ `F` (shape `[N, C]`). They will only be used for the final part of the assignment (Part 3)
* Name of each category is provided in the list `categories` (length `[C]`).

In [89]:
A = sp.load_npz('A.npz')
F = np.load('F.npy')
categories = np.load('categories.npy', allow_pickle=True).tolist()

In [3]:
assert A.shape[0] == F.shape[0]
assert F.shape[1] == len(categories)

In [4]:
print(f'The adjacency matrix is {"symmetric" if (A != A.T).sum() == 0 else "asymmetric"}')

The adjacency matrix is symmetric


# 1. Implementing spectral clustering (35 pt)
## 1.1. Construct the graph Laplacian (10 pt)

First, we need to construct the Laplacian for the given graph (*Do only use sparse operations, see [Scipy Sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html)*). 

Given the **adjacency matrix** $A \in \mathbb{R}^{N \times N},$ we define the **degree matrix** $D \in \mathbb{R}^{N \times N}$ of an undirected graph as
$$D_{ij} =  \begin{cases}\sum_{k=1}^N A_{ik} & if \;\; i = j\\ 0 & if \;\; i \ne j\end{cases}$$

If our goal is to minimize the **ratio cut**, we will need to use the **unnormalized Laplacian**, defined as
$$L_{unnorm} = D - A.$$

If our goal is to minimze the **normalized cut**, we will need to use the **normalized Laplacian** (a.k.a. symmetrized Laplacian), defined as
$$L_{sym} = I - D^{-1/2}AD^{-1/2}$$

In [23]:
row = np.array([1, 0, 1, 2, 2, 2])
col = np.array([0, 1, 2, 0, 1, 2])
data = np.array([2, 2, 3, 0, 3, 6])
M=sp.csr_matrix((data, (row, col)), shape=(4, 4))
#csr_matrix((data, (row, col)), shape=(3, 3)).toarray()
M.toarray()
M.shape[1]

4

In [24]:
(M @ M).toarray()
np.sum(M, axis=1)
np.array(range(3))
np.sqrt(np.array(range(3)))
t=np.array(range(3)).reshape(1,-1)
t.T
t * np.array([[1,2,3],[1,2,3],[1,2,3]])
(M * M).toarray()

array([[ 4,  0,  6,  0],
       [ 0, 13, 18,  0],
       [ 6, 18, 45,  0],
       [ 0,  0,  0,  0]], dtype=int64)

In [45]:
M.toarray()
construct_laplacian(M, norm_laplacian=False).asfptype().toarray()

array([[ 2., -2.,  0.,  0.],
       [-2.,  5., -3.,  0.],
       [ 0., -3.,  3.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [ ]:
K=construct_laplacian(M, norm_laplacian=False).asfptype().toarray()
t=eigsh(K,k=3,which='SM')
t[1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in true_divide


array([[-4.13228385e-01, -4.03206691e-01, -7.65055324e-01],
       [-4.13228385e-01, -4.03206691e-01,  1.35509923e-01],
       [-4.13228385e-01, -4.03206691e-01,  6.29545401e-01],
       [-6.98374474e-01,  7.15732557e-01,  1.14459432e-16]])

In [ ]:
A=M
N = A.shape[0]
diag = np.sum(A, axis=1).reshape(-1)
inv = np.sqrt(1 / diag)
row = np.array(range(N))
col = np.array(range(N))
sp.csr_matrix((row, (row, col)), shape=(N, N)).toarray()
#np.asarray(diag).reshape(-1)
diag

matrix([[2, 5, 9]])

In [34]:
def construct_laplacian(A: sp.csr_matrix, norm_laplacian: bool) -> sp.csr_matrix:
    """Construct Laplacian of a graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    norm_laplacian : bool
        Whether to construct the normalized graph Laplacian or not.
        If True, construct the normalized (symmetrized) Laplacian, L = I - D^{-1/2} A D^{-1/2}.
        If False, construct the unnormalized Laplacian, L = D - A.
        
    Returns
    -------
    L : scipy.sparse.csr_matrix, shape [N, N]
        Laplacian of the graph.
        
    """
    ##########################################################
    # YOUR CODE HERE
    N = A.shape[0]
    diag = np.asarray(np.sum(A, axis=1)).reshape(-1)
    #inv = np.sqrt(1 / diag)
    inv = [1/x if x!= 0 else 0 for x in diag]
    L = sp.csr_matrix((diag, (np.array(range(N)), np.array(range(N)))), shape=(N, N)) - A
    if norm_laplacian:
      tmp = sp.csr_matrix((inv, (np.array(range(N)), np.array(range(N)))), shape=(N, N))
      L = tmp * L * tmp
    ##########################################################
    return L

## 1.2. Spectral embedding (10 pt)

Now, we have to compute the spectral embedding for the given graph.

In order to partition the graph into $k$ clusters, such that the desired cut (ratio or normalized) is minimized, we need to consider the $k$ eigenvectors corresponding to the $k$ smallest eigenvalues of the graph Laplacian.

Since the Laplacian matrix is sparse and symmetric, we can use the function `eigsh` from the `scipy.sparse.linalg` package in order to find eigendecomposition of $L$ (`eig` - eigendecomposition, `s` - sparse, `h`- Hermitian).
The function `eigsh` directly allows you to find the smallest / largest eigenvalues by specifying the `k` and `which` parameters. 

Keep in mind that the Laplacian matrix is always positive semi-definite when picking the appropriate value for the `which` parameter.

In [6]:
from scipy.sparse.linalg import eigsh

In [ ]:
help(eigsh)

Help on function eigsh in module scipy.sparse.linalg.eigen.arpack.arpack:

eigsh(A, k=6, M=None, sigma=None, which='LM', v0=None, ncv=None, maxiter=None, tol=0, return_eigenvectors=True, Minv=None, OPinv=None, mode='normal')
    Find k eigenvalues and eigenvectors of the real symmetric square matrix
    or complex hermitian matrix A.
    
    Solves ``A * x[i] = w[i] * x[i]``, the standard eigenvalue problem for
    w[i] eigenvalues with corresponding eigenvectors x[i].
    
    If M is specified, solves ``A * x[i] = w[i] * M * x[i]``, the
    generalized eigenvalue problem for w[i] eigenvalues
    with corresponding eigenvectors x[i].
    
    Parameters
    ----------
    A : ndarray, sparse matrix or LinearOperator
        A square operator representing the operation ``A * x``, where ``A`` is
        real symmetric or complex hermitian. For buckling mode (see below)
        ``A`` must additionally be positive-definite.
    k : int, optional
        The number of eigenvalues and eige

In [7]:
def spectral_embedding(A: sp.csr_matrix, num_clusters: int, norm_laplacian: bool) -> np.array:
    """Compute spectral embedding of nodes in the given graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    num_clusters : int
        Number of clusters to detect in the data.
    norm_laplacian : bool, default False
        Whether to use the normalized graph Laplacian or not.
        
    Returns
    -------
    embedding : np.array, shape [N, num_clusters]
        Spectral embedding for the given graph.
        Each row represents the spectral embedding of a given node.
    
    """
    if (A != A.T).sum() != 0:
        raise ValueError("Spectral embedding doesn't work if the adjacency matrix is not symmetric.")
    if num_clusters < 2:
        raise ValueError("The clustering requires at least two clusters.")
    if num_clusters > A.shape[0]:
        raise ValueError(f"We can have at most {A.shape[0]} clusters (number of nodes).")
    ##########################################################
    # YOUR CODE HERE
    L = construct_laplacian(A, norm_laplacian=norm_laplacian).asfptype().toarray()
    eigenvalues, eigenvectors = eigsh(L, k=num_clusters)
    ##########################################################
    return eigenvectors

## 1.3. Determine the clusters based on the spectral embedding (15 pt)

You should use the K-means algorithm for assigning nodes to clusters, once the spectral embedding is computed.

One thing you should keep in mind, is that when using the **normalized Laplacian**, the rows of the embedding matrix **have to** be normalized to have unit $L_2$ norm. The justification of this approach is not trivial. If you are interested, the paper "A tutorial on spectral clustering" by Ulrike von Luxburg. Section 7 covers a justification involving Perturbation Theory. 

In [ ]:
X = np.array([[1, 2], [1, 4], [1, 0],
...               [10, 2], [10, 4], [10, 0]])
X

array([[ 1,  2],
       [ 1,  4],
       [ 1,  0],
       [10,  2],
       [10,  4],
       [10,  0]])

In [8]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

In [9]:
def spectral_clustering(A: sp.csr_matrix, num_clusters: int, norm_laplacian: bool, seed: int = 42) -> np.array:
    """Perform spectral clustering on the given graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    num_clusters : int
        Number of clusters to detect in the data.
    norm_laplacian : bool, default False
        Whether to use the normalized graph Laplacian or not.
    seed : int, default 42
        Random seed to use for the `KMeans` clustering.
        
    Returns
    -------
    z_pred : np.array, shape [N]
        Predicted cluster indicators for each node.
        
    """
    model = KMeans(num_clusters, random_state=seed)
    ##########################################################
    # YOUR CODE HERE
    L = construct_laplacian(A, norm_laplacian=norm_laplacian).asfptype().toarray()
    eigenvalues, eigenvectors = eigsh(L, k=num_clusters)
    if norm_laplacian:
      eigenvectors = normalize(eigenvectors, norm='l2')
    kmeans = model.fit(eigenvectors)
    z_pred = kmeans.labels_
    ##########################################################
    return z_pred

# 2. Quantitatively evaluate the results (10 pt)

In [10]:
def labels_to_list_of_clusters(z: np.array) -> List[List[int]]:
    """Convert predicted label vector to a list of clusters in the graph.
    This function is already implemented, nothing to do here.
    
    Parameters
    ----------
    z : np.array, shape [N]
        Predicted labels.
        
    Returns
    -------
    list_of_clusters : list of lists
        Each list contains ids of nodes that belong to the same cluster.
        Each node may appear in one and only one partition.
    
    Examples
    --------
    >>> z = np.array([0, 0, 1, 1, 0])
    >>> labels_to_list_of_clusters(z)
    [[0, 1, 4], [2, 3]]
    
    """
    return [np.where(z == c)[0] for c in np.unique(z)]

## 2.1. Compute ratio cut (5 pt)

Your task is to implement functions for computing the **ratio cut** and **normalized cut** for a given partition.

Ratio cut and normalized cut are defined on the slide 14 of the lecture slides.


The function `labels_to_list_of_clusters` can be helpful here.

In [ ]:
x=[[0, 1, 4], [2, 3]]
len(x)
x[0]
A.toarray()
np.sum(A[[1,2],])

14

In [46]:
def compute_ratio_cut(A: sp.csr_matrix, z: np.array) -> float:
    """Compute the ratio cut for the given partition of the graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    z : np.array, shape [N]
        Cluster indicators for each node.
    
    Returns
    -------
    ratio_cut : float
        Value of the cut for the given partition of the graph.
        
    """
    
    ##########################################################
    # YOUR CODE HERE
    partition = labels_to_list_of_clusters(z)
    ratio_cut = 0
    for item in partition:
      ratio_cut += (np.sum(A[item, :]) - np.sum(A[item, item])) / len(item)
    ##########################################################
    return ratio_cut

## 2.2. Compute normalized cut (5 pt)

**Important**: if a cluster only contains a single node, define its volume to be 1 to avoid division by zero errors.

In [84]:
def compute_normalized_cut(A: sp.csr_matrix, z: np.array) -> float:
    """Compute the normalized cut for the given partition of the graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    z : np.array, shape [N]
        Cluster indicators for each node.
    
    Returns
    -------
    norm_cut : float
        Value of the normalized cut for the given partition of the graph.
        
    """
    
    ##########################################################
    # YOUR CODE HERE
    partition = labels_to_list_of_clusters(z)
    norm_cut = 0
    for item in partition:
      norm_cut += (np.sum(A[item, :]) - np.sum(A[item, item])) / max(np.sum(A[item, :]), 1)
    ##########################################################
    return norm_cut

Notice, how using the unnormalized Laplacian leads to a much better ratio cut, while the normalized Laplacian leads to better normalized cut.

In [13]:
num_clusters = 6

In [91]:
np.random.seed(12903)
norm_laplacian = False
z_unnorm = spectral_clustering(A, num_clusters, norm_laplacian)
print('When using L_unnorm:')
print(' ratio cut = {:.3f}'.format(compute_ratio_cut(A, z_unnorm)))
print(' normalized cut = {:.3f}'.format(compute_normalized_cut(A, z_unnorm)))
print(' sizes of partitions are: {}'.format([len(clust) for clust in labels_to_list_of_clusters(z_unnorm)]))

When using L_unnorm:
 ratio cut = 58837.277
 normalized cut = 6.000
 sizes of partitions are: [3379, 1, 1, 1, 1, 1]


In [90]:
np.random.seed(12323)
norm_laplacian = True
z_norm = spectral_clustering(A, num_clusters, norm_laplacian)
print('When using L_norm:')
print(' ratio cut = {:.3f}'.format(compute_ratio_cut(A, z_norm)))
print(' normalized cut = {:.3f}'.format(compute_normalized_cut(A, z_norm)))
print(' sizes of partitions are: {}'.format([len(clust) for clust in labels_to_list_of_clusters(z_norm)]))

When using L_norm:
 ratio cut = 8228.430
 normalized cut = 6.000
 sizes of partitions are: [977, 153, 805, 310, 583, 556]


# 3. Visualize the results (5 pt)

In the final part of the assignment, your task is to print out the 5 most popular types of restaurants visited by the users in each cluster.

In [82]:
def print_top_categories_for_each_cluster(top_k: int, z: np.array, F: sp.csr_matrix, categories: List[str]):
    """Print the top-K categories among users in each cluster.
    For each cluster, the function prints names of the top-K categories,
    and number of users that like the respective category (separated by a comma).
    The function doesn't return anything, just prints the output.
    
    Parameters
    ----------
    top_k : int
        Number of most popular categories to print for each cluster.
    z : np.array, shape [N]
        Cluster labels.
    F : sp.csr_matrix, shape [N, C]
        Matrix that tells preferences of each user to each category.
        F[i, c] = 1 if user i gave at least one positive review to at least one restaurant in category c.
    categories : list, shape [C]
        Names of the categories.
        
    """
    ##########################################################
    # YOUR CODE HERE
    partition = labels_to_list_of_clusters(z)
    for i, item in enumerate(partition):
      rowsum = np.sum(F[item, :], axis=0)
      indices = np.argsort(-rowsum)
      print('Most popular categories in cluster ' + str(i))
      for k in range(top_k):
        print(' - ' + str(categories[indices[k]]) + ', ' + str(int(rowsum[indices[k]])))
      print()
    ##########################################################

In [73]:
partition = labels_to_list_of_clusters(z_norm)
for i, item in enumerate(partition):
  rowsum = np.sum(F[item, ], axis=0)
  indices = np.argsort(-rowsum)

In [77]:
np.sum(np.array([[2,3],[4,5],[6,7]]), axis=0)

array([12, 15])

In [ ]:
print('Most popular categories in cluster ' + str(4))
print(' - Seafood, ' + str(315))
print(' ')
print('Most popular categories in cluster ' + str(4))
print(6)
print()
np.argsort(-np.array([3,2,4]))

Most popular categories in cluster 4
 - Seafood, 315
 
Most popular categories in cluster 4
6



array([2, 0, 1])

In [83]:
np.random.seed(23142)
z_norm = spectral_clustering(A, num_clusters, True)
r = print_top_categories_for_each_cluster(5, z_norm, F, categories)

Most popular categories in cluster 0
 - Breakfast & Brunch, 816
 - Japanese, 751
 - Sandwiches, 689
 - Italian, 671
 - Coffee & Tea, 638

Most popular categories in cluster 1
 - Breakfast & Brunch, 135
 - Italian, 131
 - Japanese, 118
 - Pizza, 115
 - Sandwiches, 115

Most popular categories in cluster 2
 - Japanese, 652
 - Breakfast & Brunch, 636
 - Italian, 580
 - Asian Fusion, 527
 - Sandwiches, 520

Most popular categories in cluster 3
 - Breakfast & Brunch, 268
 - Sandwiches, 253
 - Italian, 252
 - Japanese, 236
 - American (Traditional), 228

Most popular categories in cluster 4
 - Breakfast & Brunch, 473
 - Japanese, 428
 - Italian, 416
 - Sandwiches, 407
 - Cafes, 353

Most popular categories in cluster 5
 - Breakfast & Brunch, 466
 - Italian, 415
 - Sandwiches, 389
 - Japanese, 375
 - Pizza, 348



In [58]:
labels_to_list_of_clusters(z_norm)

[array([   1,    2,    3,    5,    7,    9,   11,   15,   16,   21,   22,
          23,   30,   31,   34,   35,   41,   43,   47,   51,   52,   55,
          56,   63,   66,   73,   78,   80,   84,   87,   89,   93,   97,
         103,  108,  110,  113,  120,  125,  129,  131,  138,  151,  158,
         160,  168,  171,  175,  177,  179,  180,  182,  186,  190,  197,
         198,  200,  202,  203,  210,  213,  215,  218,  219,  225,  229,
         233,  235,  236,  241,  242,  245,  246,  247,  248,  254,  257,
         262,  268,  272,  280,  282,  283,  289,  291,  293,  305,  307,
         308,  312,  313,  315,  319,  321,  324,  330,  333,  334,  336,
         343,  345,  348,  354,  355,  361,  364,  373,  377,  380,  382,
         383,  391,  392,  394,  395,  404,  405,  408,  413,  417,  423,
         425,  429,  432,  436,  438,  439,  450,  453,  456,  457,  461,
         463,  465,  470,  475,  476,  480,  482,  486,  494,  496,  497,
         501,  503,  512,  513,  516, 